In [1]:
import k3d

In [2]:
import numpy as np

from pathlib import Path
import imageio.v2 as imageio

from hc3d.vis import CameraCone
from hc3d.render import compute_intrinsics, unproject
from hc3d.utils import batch_img_resize

In [8]:
import warnings
warnings.filterwarnings("error")

In [3]:
def load_imgs():
    # root = Path("/scratch/omni3d_data/ml-hypersim/hypersim/ai_001_001/images/scene_cam_00_final_preview")
    root = Path("/scratch/omni3d_data/ml-hypersim/hypersim/ai_001_008/images/scene_cam_00_final_preview")
    # example fname frame.0001.tonemap.jpg
    fnames = sorted(list(root.glob("*.jpg")))
    imgs = [imageio.imread(str(f)) for f in fnames]
    return np.array(imgs)

In [4]:
poses = np.load("/home/whc/projects/omni3d/008_poses.npy")
print(poses.shape)
n = len(poses)

(100, 4, 4)


In [5]:
H, W = 768, 1024
fov = 60
K = compute_intrinsics(W / H, fov, H)

In [6]:
imgs = load_imgs()
print(imgs.shape)
imgs = batch_img_resize(imgs, new_h=200)
print(imgs.shape)

(100, 768, 1024, 3)
(100, 200, 266, 3)


In [7]:
def generate_cam(po, color, im=None):
    cone = CameraCone(K, po, W, H, scale=10.0,
                      top_left_corner=(0, 0), color=color)
    lset = cone.as_k3d_lineset()
    if im is None:
        return [lset]
    else:
        # o3d img tsr requires contiguous array
        im = np.ascontiguousarray(im)
        view_plane = cone.as_k3d_viewplane(im)
        return [lset, view_plane]

In [9]:
geoms = []
for i in range(n):
    po = poses[i]
    im = imgs[i]
    geoms.extend(
        generate_cam(po, [200, 20, 200], im=im)
    )

In [10]:
plot = k3d.plot()
plot.display()

Output()

In [11]:
for elem in geoms:
    plot += elem

In [12]:
plot.close()

In [46]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(imgs[0])